<a href="https://colab.research.google.com/github/nakamura196/NDLOCR-GoogleColabVersion/blob/main/NDLOCR_googlecolabversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. GPUの情報を確認する

In [1]:
!nvidia-smi

Tue Jul 25 06:38:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)

In [2]:
!git clone --recursive --branch ver.1 https://github.com/ndl-lab/ndlocr_cli

Cloning into 'ndlocr_cli'...
remote: Enumerating objects: 416, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 416 (delta 143), reused 158 (delta 81), pack-reused 144
Receiving objects: 100% (416/416), 203.93 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (215/215), done.
Submodule 'src/deskew_HT' (https://github.com/ndl-lab/deskew_HT.git) registered for path 'src/deskew_HT'
Submodule 'src/ndl_layout' (https://github.com/ndl-lab/ndl_layout.git) registered for path 'src/ndl_layout'
Submodule 'src/separate_pages_ssd' (https://github.com/ndl-lab/separate_pages_ssd.git) registered for path 'src/separate_pages_ssd'
Submodule 'src/text_recognition' (https://github.com/ndl-lab/text_recognition.git) registered for path 'src/text_recognition'
Cloning into '/content/ndlocr_cli/src/deskew_HT'...
remote: Enumerating objects: 41, done.        
remote: Counting objects: 100% (41/41), done.        
remote: Compressing objects: 100

# 2. 必要なパッケージをインストールする

In [3]:
PROJECT_DIR="/content/ndlocr_cli"

In [4]:
!pip install mmcv==2.0.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html
!pip install mmdet==3.0.0
!pip install lmdb

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 4.3 MB/s eta 0:00:00


# 4. OCRに必要な学習済みモデルをダウンロードする

In [5]:
%cd {PROJECT_DIR}
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools
%cd /content/

/content/ndlocr_cli
--2023-07-25 06:38:58--  https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt
Resolving lab.ndl.go.jp (lab.ndl.go.jp)... 18.64.183.3, 18.64.183.19, 18.64.183.107, ...
Connecting to lab.ndl.go.jp (lab.ndl.go.jp)|18.64.183.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28088 (27K) [text/plain]
Saving to: ‘./src/text_recognition/models/mojilist_NDL.txt’

mojilist_NDL.txt    100%[===================>]  27.43K  --.-KB/s    in 0.002s  

2023-07-25 06:38:58 (11.7 MB/s) - ‘./src/text_recognition/models/mojilist_NDL.txt’ saved [28088/28088]

--2023-07-25 06:38:58--  https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth
Resolving lab.ndl.go.jp (lab.ndl.go.jp)... 18.64.183.3, 18.64.183.19, 18.64.183.107, ...
Connecting to lab.ndl.go.jp (lab.ndl.go.jp)|18.64.183.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191768312 (183M) [application/x-www-form-urlencoded]
Saving to: ‘./s

# 5. 環境変数を追加する

In [6]:
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

# 6. PDFを画像に変換するためのパッケージのインストール

In [7]:
!apt-get install poppler-utils
!pip install pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.1 [186 kB]
Fetched 186 kB in 1s (234 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 129824 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.1_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.1) ...
Setting up poppler-utils (22.02.0-2ubuntu0.1) ...
Processing triggers for man-db (2.10.2-1) ...


# 7. テキスト化したいPDFをダウンロードする
今回は、ROIS-DS人文学オープンデータ共同利用センター(http://codh.rois.ac.jp/
)が提供している

近代雑誌データセット　http://codh.rois.ac.jp/modern-magazine/
から、

東洋学芸雑誌(https://dglb01.ninjal.ac.jp/ninjaldl/bunken.php?title=toyogakuge)

第一号(https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf)

をダウンロードしてみます。

In [8]:
!curl --insecure https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf -o /content/tygz-001.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.1M  100 14.1M    0     0  6436k      0  0:00:02  0:00:02 --:--:-- 6435k


# 8. PDFをjpeg画像に変換する

In [9]:
from pathlib import Path
from pdf2image import convert_from_path
import os
pdf_path = Path("/content/tygz-001.pdf")

input_dir = "/content/tygz-001"

os.makedirs(f"{input_dir}/img",exist_ok=True)
img_path=Path(f"{input_dir}/img")

convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg',output_file=pdf_path.stem,dpi=100)

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8AB1000>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23910>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23970>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23A00>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23A60>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23AC0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23B20>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23B80>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23BE0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23C40>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7893C8B23CA0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB siz

# 9. OCRの実行

/content/tygz-001以下のimgディレクトリ内の画像を処理し、
/content/tygz-001_outputに出力する場合

In [10]:
%cd {PROJECT_DIR}
!python main.py infer {input_dir} {input_dir}_output -s s -x

/content/ndlocr_cli
start inference !
input_root : /content/tygz-001
output_root : /content/tygz-001_output
config_file : config.yml
load from config=src/ndl_layout/models/ndl_layout_config.py, checkpoint=src/ndl_layout/models/epoch_140_all_eql_bt.pth
Loads checkpoint by local backend from path: src/ndl_layout/models/epoch_140_all_eql_bt.pth
No Transformation module specified
No SequenceModeling module specified
model input parameters 32 1200 20 1 512 256 7085 100 None ResNet None CTC
loading pretrained model from src/text_recognition/models/ndlenfixed64-mj0-synth1.pth
[{'input_dir': '/content/tygz-001', 'img_list': ['/content/tygz-001/img/tygz-0010001-01.jpg', '/content/tygz-001/img/tygz-0010001-02.jpg', '/content/tygz-001/img/tygz-0010001-03.jpg', '/content/tygz-001/img/tygz-0010001-04.jpg', '/content/tygz-001/img/tygz-0010001-05.jpg', '/content/tygz-001/img/tygz-0010001-06.jpg', '/content/tygz-001/img/tygz-0010001-07.jpg', '/content/tygz-001/img/tygz-0010001-08.jpg', '/content/tygz-

# 10. 結果の確認

In [11]:
import glob
import os
for fpath in sorted(glob.glob(f"{input_dir}_output/tygz-001/txt/*_main.txt")):
    with open(fpath) as f:
        txtdata=f.read()
        print(os.path.basename(fpath).replace("_main.txt",""))
        print(txtdata)

tygz-0010001-01
明治十六年九月再版
東法學藝羅誌


tygz-0010001-02



tygz-0010001-03
ホ或ハ此雜誌ノ讀三難キヲ困シムモノ
ヘ甘苦相半ナラシメ以テ世人ノ望二負
上二渉レル平易ナル文章ヲモ其間ニ雜
ク無キヲ期スト云爾
ナキ二非ス因テ更二其區域ヲ廣メ文藝
トノ三ヲ討論スル二非スト〓トモ世尚

常ニ憂フルトコロナリ故ニ之ヲ救ハン
我邦人ノ理學ノ思想二乏シキハ識者ノ
カ爲ニ此雜誌二理學二關係アル文章ヲ
掲載シテ其性質及ヒ功用ヲ世ニ明二七
緒言


tygz-0010001-04



tygz-0010001-05
○理學ノ快樂一二二、
○平賀源内傳片山沖堂一一五、
○非時事小言論上田秀成
○答東京經濟雜誌井上哲次郎一一〇、
三三、八四
一五八、
○ハムレット(反譯)尚今居士一一七、
○本誌改良ノ因由
皇帝陛下ノ万歳
ヲ述ヘテ
○羅馬字ヲ以テ日
一〇六、一四二
本語ヲ綴ルノ説
○理學ノ快樂
○遊日光山雜記谷田部梅吉報三八、六四、
ヲ祝シ奉ル
○水ヲ燃料ニ供ス
○度量衡説附貨幣鮫島晋
失矢田部良吉一二七、一五
〇一、一三六
上田秀成四七、
一三六

加藤弘之一、一六、
論説之部
論説人名丁數
テ人才ヲ得ルノ
○世界開闢〓論隈本有尚五六、
○分子ノ解磯野徳三郎七二、
四九、七一、
○人爲淘汰ニヨリ
術ヲ論ス
九一
○西哲叢談シヨン、ダルトン六五、
○題有機化學沿革
評スルヲ評ス
○泰西人ノ孔子ヲ
〇)〓
○西哲叢談ジヨン、ダルトン
○ヒューム氏自由
史後
○ヒューム氏自由
〓
井上哲二郎五三
シテハ如何ヲ論九一
○以ヲ〓子ヲ井上哲二郎五三、


tygz-0010001-06
樫村清徳二一二、
○文章論小中村清矩二〇九、
○理學功用辨杉浦重剛二一一
○支那紙幣史畧平沼淑郎一六〇、一八八
尚今居士二二三、
二二七、
○職業教育論手島精一二一四、
○人類ノ紀元松下丈吉一五五、一七九
○書ハ美術ナラス小山正太郎
ヲ評ス
一七二、二〇五、
○カムプベル氏英
ノ一因タリ抑モ
定スル論
○抜刀隊ノ歌外山正一一七〇
○岩垣月洲傳杉浦正臣一六九
鈎玄堂主人一六七
○讀原道評杉浦正臣二〇三、
國海軍の詩
○クック氏ノ演説
○謠曲ハ肺氣腫症
之ヲ廢棄セン歟
○謠曲ハ肺氣腫症
之ヲ廢棄セン歟

○讀韓氏原道井上哲次郎一三九、
中川元一八一
井上哲二